In [197]:
from reedsolo import RSCodec
import base64
import json

In [198]:


class ReedSolomon:
    def __init__(self, ecc_symbols):
        self.ecc_symbols = ecc_symbols
        self.rsc = RSCodec(ecc_symbols)
    
    def encode(self, data: str, verbose: bool = False) -> bytearray:
        data_bytes = data.encode('utf-8')
        encoded = self.rsc.encode(data_bytes)
        if verbose:
            print(data, "- исходный текст")
            print(list(data_bytes), "- в байтах")
            print(list(encoded), "- код Рида-Соломона")
        return encoded
    
    def fix_errors(self, encoded_data: bytearray, verbose: bool = False) -> tuple[bytearray, int]:
        try:
            decoded, decoded_bytes, errata_pos = self.rsc.decode(encoded_data)
            errors_fixed = len(errata_pos) > 0
            encoded = self.rsc.encode(decoded)
            if verbose:
                print(list(encoded_data), "- до исправления ошибок")
                print(list(encoded), "- после исправления ошибок")
            return encoded, errors_fixed
            
        except Exception:
            return None, False, -1
    
    def decode(self, encoded_data: bytearray, verbose: bool = False) -> str:
        decoded, *_ = self.rsc.decode(encoded_data)
        decoded_str = decoded.decode('utf-8')
        if verbose:
            print(list(encoded_data), "- код Рида-Соломона")
            print(list(decoded), "- декодированная строка")
            print(decoded_str, "- перевели обратно в текст")
        return decoded_str


    def simulate_errors(self, encoded_data: bytearray, error_positions: list[int], verbose: bool = False) -> bytearray:
        corrupted = bytearray(encoded_data)
        
        for pos in error_positions:
            if pos < len(corrupted):
                # Инвертируем бит в указанной позиции
                corrupted[pos] ^= 0xFF  # Меняем все биты байта
        
        broken_data = bytearray(corrupted)
        if verbose:
            print(list(encoded_data), "- код Рида-Соломона")
            print(list(broken_data), "- поломанный код Рида-Соломона")
        return broken_data


In [199]:
rs = ReedSolomon(ecc_symbols=2)  # ecc_symbols//2 ошибок

In [200]:
g = {k:list(v) for k, v in rs.rsc.gen.items()}
print(g)

{2: [1, 3, 2]}


In [201]:
print(list(rs.rsc.gf_exp))

[1, 2, 4, 8, 16, 32, 64, 128, 29, 58, 116, 232, 205, 135, 19, 38, 76, 152, 45, 90, 180, 117, 234, 201, 143, 3, 6, 12, 24, 48, 96, 192, 157, 39, 78, 156, 37, 74, 148, 53, 106, 212, 181, 119, 238, 193, 159, 35, 70, 140, 5, 10, 20, 40, 80, 160, 93, 186, 105, 210, 185, 111, 222, 161, 95, 190, 97, 194, 153, 47, 94, 188, 101, 202, 137, 15, 30, 60, 120, 240, 253, 231, 211, 187, 107, 214, 177, 127, 254, 225, 223, 163, 91, 182, 113, 226, 217, 175, 67, 134, 17, 34, 68, 136, 13, 26, 52, 104, 208, 189, 103, 206, 129, 31, 62, 124, 248, 237, 199, 147, 59, 118, 236, 197, 151, 51, 102, 204, 133, 23, 46, 92, 184, 109, 218, 169, 79, 158, 33, 66, 132, 21, 42, 84, 168, 77, 154, 41, 82, 164, 85, 170, 73, 146, 57, 114, 228, 213, 183, 115, 230, 209, 191, 99, 198, 145, 63, 126, 252, 229, 215, 179, 123, 246, 241, 255, 227, 219, 171, 75, 150, 49, 98, 196, 149, 55, 110, 220, 165, 87, 174, 65, 130, 25, 50, 100, 200, 141, 7, 14, 28, 56, 112, 224, 221, 167, 83, 166, 81, 162, 89, 178, 121, 242, 249, 239, 195, 155, 4

In [202]:
print(list(rs.rsc.gf_log))

[0, 0, 1, 25, 2, 50, 26, 198, 3, 223, 51, 238, 27, 104, 199, 75, 4, 100, 224, 14, 52, 141, 239, 129, 28, 193, 105, 248, 200, 8, 76, 113, 5, 138, 101, 47, 225, 36, 15, 33, 53, 147, 142, 218, 240, 18, 130, 69, 29, 181, 194, 125, 106, 39, 249, 185, 201, 154, 9, 120, 77, 228, 114, 166, 6, 191, 139, 98, 102, 221, 48, 253, 226, 152, 37, 179, 16, 145, 34, 136, 54, 208, 148, 206, 143, 150, 219, 189, 241, 210, 19, 92, 131, 56, 70, 64, 30, 66, 182, 163, 195, 72, 126, 110, 107, 58, 40, 84, 250, 133, 186, 61, 202, 94, 155, 159, 10, 21, 121, 43, 78, 212, 229, 172, 115, 243, 167, 87, 7, 112, 192, 247, 140, 128, 99, 13, 103, 74, 222, 237, 49, 197, 254, 24, 227, 165, 153, 119, 38, 184, 180, 124, 17, 68, 146, 217, 35, 32, 137, 46, 55, 63, 209, 91, 149, 188, 207, 205, 144, 135, 151, 178, 220, 252, 190, 97, 242, 86, 211, 171, 20, 42, 93, 158, 132, 60, 57, 83, 71, 109, 65, 162, 31, 45, 67, 216, 183, 123, 164, 118, 196, 23, 73, 236, 127, 12, 111, 246, 108, 161, 59, 82, 41, 157, 85, 170, 251, 96, 134, 177, 

In [203]:
original_text = "hello"
encoded = rs.encode(original_text, verbose=True)

hello - исходный текст
[104, 101, 108, 108, 111] - в байтах
[104, 101, 108, 108, 111, 236, 142] - код Рида-Соломона


In [204]:
encoded = rs.simulate_errors(encoded, error_positions=[0], verbose=True)

[104, 101, 108, 108, 111, 236, 142] - код Рида-Соломона
[151, 101, 108, 108, 111, 236, 142] - поломанный код Рида-Соломона


In [205]:
encoded, errors_fixed = rs.fix_errors(encoded, verbose=True)

[151, 101, 108, 108, 111, 236, 142] - до исправления ошибок
[104, 101, 108, 108, 111, 236, 142] - после исправления ошибок


In [206]:
decoded = rs.decode(encoded, verbose=True)

[104, 101, 108, 108, 111, 236, 142] - код Рида-Соломона
[104, 101, 108, 108, 111] - декодированная строка
hello - перевели обратно в текст
